# Chatbot

## 1. First, make some data preprocessing to clean up the data. You can use your solution to the assignment of the data preprocessing checkpoint of this module.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
import spacy
import re
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement
import random

import warnings
warnings.filterwarnings(action="ignore")

In [6]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'cornell_movie_dialogs'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

dialogs_df = pd.read_sql_query('select * from dialogs',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [7]:
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done below, so it may take a while
dialogs_doc = nlp(" ".join(dialogs_df.dialogs))

In [13]:
dialog_sents = [sent.text for sent in dialogs_doc.sents if len(sent.text) > 1]

## 2. Develop a chatbot using this corpus. In doing this, you're free to choose a chatbot development libary like ChatterBot or to write your own code from scratch.

In [14]:
GREETING_INPUTS = ["hello", "hi", "greetings", "what's up","hey"]
GREETING_RESPONSES = ["hello", "hi", "hey", "hi there"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [1]:
# create a chatbot
chatbot = ChatBot('Dialogs')
# this is to remove the accumulated knowledge base
chatbot.storage.drop()

# create a new trainer for the chatbot
trainer = ListTrainer(chatbot)

# train the chatbot
trainer.train(dialog_sents)

## 3. Make a conversation with your chatbot and discuss the strengths and weaknesses of it.

In [0]:
import random

In [13]:
print("Cornell: I will try to respond you reasonably. If you want to exit, type bye please.")

# below is the chatting
while True:
    
    user_input = input("User: ")
    user_input=user_input.lower()
    
    if(user_input!='bye'):
        if(user_input == 'thanks' or user_input == 'thank you'):
            break
            print("Cornell: You're welcome.")
        else:
            if(greeting(user_input) != None):
                print("Cornell: " + greeting(user_input))
            else:
                print("Cornell: ", end = "")
                print(chatbot.get_response(user_input))
    else:
        print("Cornell: Bye! It was a great chat.")
        break

Cornell: I will try to respond you reasonably. If you want to exit, type bye please.
User: Hello
Cornell: hello
User: Do you enjoy watching movies?
Cornell: Made me sick, but what did I care?
User: Oh! Well, how are you then?
Cornell: I just struck it rich... I think we can do a little business.
User: What kind of business?
Cornell: Black market art.
User: Oh no. I prefer talking about movies.
Cornell: Why?
User: Because, I'm kind of a descent person and like to respect the rule of law.
Cornell: Who?
User: Me.
Cornell: This endless ...blonde babble.
User: Anyway, do you know something about technology?
Cornell: I believe we share an art instructor Have fun tonight?
User: No. What about artificial intelligence?
Cornell: Something, apperently, of an unknown origin.
User: Definitely.
Cornell: I heard what happened.
User: What happened?
Cornell: We and my pal here have been taking it on the chin for the last few hours... You real down?
User: Hmm, I think I need to go now.
Cornell: Well, no

We have some odd hits where dialogue approximates coherence, but generally it seems like the model requires a larger dataset or more tuning.

Ran into some issues with spacy dependencies, will discuss with mentor.